In [1]:
import httpx
import pandas
import tempfile
import bw2data as bd
import math
import country_converter as coco

In [2]:
df = pandas.read_excel(
    "/Users/cmutel/Downloads/AWARE_country_regions_Corrected_online_20230113-1.xlsx", 
    keep_default_na=False,
    na_values=[None, "", " "],
    sheet_name="AWARE-annual", 
)

In [3]:
bd.projects.set_current("agribalyse-3.1.1 transformations")

In [4]:
bd.databases

Databases dictionary with 8 object(s):
	agribalyse-3.1.1
	agribalyse-3.1.1-biosphere-placeholders
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ecoinvent-3.8-biosphere
	ecoinvent-3.8-cutoff
	generic-regionalized-biosphere-proxies
	generic_simapro_ecoinvent_biosphere_placeholders

In [5]:
consumption = [
    x 
    for x in bd.Database('ecoinvent-3.10-biosphere') 
    if x['name'].lower().startswith("water")
    and not x['name'].lower().startswith("water, salt")
    and x['categories'][0] == 'natural resource'
]
consumption

['Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')),
 'Water, in air' (cubic meter, None, ('natural resource', 'in air')),
 'Water, well, in ground' (cubic meter, None, ('natural resource', 'in water')),
 'Water, turbine use, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')),
 'Water, river' (cubic meter, None, ('natural resource', 'in water')),
 'Water, lake' (cubic meter, None, ('natural resource', 'in water')),
 'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in ground')),
 'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'fossil well')),
 'Water, cooling, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water'))]

In [6]:
release = [
    x 
    for x in bd.Database('ecoinvent-3.10-biosphere') 
    if x['name'].lower().startswith("water")
    and not x['name'].lower().startswith("water, salt")
    and x['categories'][0] != 'natural resource'
    and x['categories'][0] != 'air'
    and x['categories'] != ('water', 'ground-, long-term')
    and x['categories'] != ('water', 'fossil well')
]
release

['Water' (cubic meter, None, ('water', 'surface water')),
 'Water' (cubic meter, None, ('water', 'ground-')),
 'Water' (cubic meter, None, ('water',)),
 'Water' (cubic meter, None, ('water', 'ocean'))]

In [7]:
cfs = []
reached_zimbabwe = False

FIXES = {
    'Raunion': 'Reunion',
    'OCEANIA': 'UN-OCEANIA',
    'ENTSOE': 'ENTSO-E'
}
IGNORE = {'World AGRI', 'World NON-AGRI', 'Europe-CH', 'RoW',"BRIC","BRICS","OECD","OECD+BRIC","OECD+BRICS","REGIONS"}

for row_dict in df.to_dict(orient="records"):
    if not isinstance(row_dict['Unnamed: 0'], str) and math.isnan(row_dict['Unnamed: 0']):
        continue
    if not reached_zimbabwe:
        cfs.append((
            row_dict['Agg_CF_unspecified'],
            coco.convert(names=[FIXES.get(row_dict['Unnamed: 0'], row_dict['Unnamed: 0'])], to='ISO2'),
        ))
        if row_dict['Unnamed: 0'] == 'Zimbabwe':
            reached_zimbabwe = True
    else:
        if row_dict['Unnamed: 0'] == 'World, UNKNOWN':
            cfs.append((
                row_dict['Agg_CF_irri'],
                "GLO",
            ))
        elif row_dict['Unnamed: 0'] in IGNORE:
            continue
        else:
            cfs.append((
                row_dict['Agg_CF_unspecified'],
                FIXES.get(row_dict['Unnamed: 0'], row_dict['Unnamed: 0']),
            ))

cfs

[(57.935456987292476, 'AF'),
 (34.1634942960456, 'AL'),
 (64.17180130077834, 'DZ'),
 (4.417532544, 'AS'),
 (56.3308179373251, 'AD'),
 (5.788541614191322, 'AO'),
 (22.37208753, 'AI'),
 (5.759905111728797, 'AG'),
 (30.14148180514807, 'AR'),
 (85.75290069851204, 'AM'),
 (0.0, 'AW'),
 (71.08118653712059, 'AU'),
 (1.2459292893288103, 'AT'),
 (84.6293212973196, 'AZ'),
 (8.72716077896561, 'BH'),
 (2.9863210944027583, 'BD'),
 (9.70662042091232, 'BB'),
 (3.364175972996923, 'BY'),
 (1.3736930005596066, 'BE'),
 (1.0883962832274388, 'BZ'),
 (6.323668271898761, 'BJ'),
 (1.0214274439494864, 'BT'),
 (2.891210759394875, 'BO'),
 (1.1746002891612453, 'BA'),
 (21.712667514866858, 'BW'),
 (2.2752038902763507, 'BR'),
 (14.6191233, 'VG'),
 (0.18278962258374884, 'BN'),
 (26.6952216599752, 'BG'),
 (18.20079075155446, 'BF'),
 (29.111873606634934, 'BI'),
 (8.564984330130088, 'KH'),
 (7.14011138484715, 'CM'),
 (6.577624816701168, 'CA'),
 (24.81282098, 'CV'),
 (8.88104290530532, 'CF'),
 (21.602798326139393, 'TD')

In [8]:
aware = bd.Method(("AWARE", "unspecified", "2.1 (2023-01-13)"))
aware.register(
    unit="Availability Minus Demand (AMD), m**3/m**2/month",
    description="""AWARE is a water use midpoint indicator representing the relative Available WAter REmaining per area in a watershed, after the demand of humans and aquatic ecosystems has been met. It assesses the potential of water deprivation, to either humans or ecosystems, building on the assumption that the less water remaining available per area, the more likely another user will be deprived. It is first calculated as the water Availability Minus the Demand (AMD) of humans and aquatic ecosystems and is relative to the area (m3 m-2 month-1). In a second step, the value is normalized with the world average result (AMD = 0.0136m3m-2 month-1) and inverted, and hence represents the relative value in comparison with the average m3 consumed in the world (the world average is calculated as a consumption-weighted average). Once inverted, 1/AMD  can be interpreted as a surface-time equivalent to generate unused water in this region. The indicator is limited to a range from 0.1 to 100, with a value of 1 corresponding to the world average, and a value of 10, for example, representing a region where there is 10 times less available water remaining per area than the world average.""",
    url = "http://www.wulca-waterlca.org/aware.html"
)
aware.write(
    [
        (flow.key, score, location) 
        for flow in consumption for score, location in cfs
    ] 
    + [
        (flow.key, score * -1, location) 
        for flow in release for score, location in cfs
    ]
)

In [9]:
countries = {
    x['location']
    for x in bd.Database('ecoinvent-3.10-cutoff')
    if (len(x['location']) == 2 and x['location'].upper() == x['location'])
}

locations = {
    x['location']
    for x in bd.Database('ecoinvent-3.10-cutoff')
    if not (len(x['location']) == 2 and x['location'].upper() == x['location'])
}

In [10]:
mapping = {
    'Canada without Quebec': 'CA',
    'Europe without Austria': 'RER',
    'Europe without Switzerland and Austria': 'RER',
    'Europe without Switzerland': 'RER',
    'Europe, without Russia and Türkiye': 'RER',
    'IAI Area, Africa': 'RAF',
    'IAI Area, Asia, without China and GCC': 'RAS',
    'IAI Area, EU27 & EFTA': 'RER',
    'IAI Area, Gulf Cooperation Council': 'RME',
    'IAI Area, North America': 'RNA',
    'IAI Area, Russia & RER w/o EU27 & EFTA': 'RU',
    'IAI Area, South America': 'RLA',
    'NORDEL': 'ENTSO-E',
    'North America without Quebec': 'RNA',
    'OCEANIA': 'UN-OCEANIA',
    'RER w/o CH+DE': 'RER',
    'RER w/o DE+NL+RU': 'RER',
    'RER w/o RU': 'RER',
    'RoE': 'RER',
    'RoW': 'GLO',
    'SAS': 'RAS',
    'UCTE without Germany': 'ENTSO-E',
    'UCTE': 'ENTSO-E',
    'WECC': 'US',
    'WEU': 'RER',
}

for location in locations:
    if location in mapping:
        continue
    elif len(location) > 3 and location[2] == "-" and location[:2] in countries:
        mapping[location] = location[:2]
    else:
        continue

mapping

{'Canada without Quebec': 'CA',
 'Europe without Austria': 'RER',
 'Europe without Switzerland and Austria': 'RER',
 'Europe without Switzerland': 'RER',
 'Europe, without Russia and Türkiye': 'RER',
 'IAI Area, Africa': 'RAF',
 'IAI Area, Asia, without China and GCC': 'RAS',
 'IAI Area, EU27 & EFTA': 'RER',
 'IAI Area, Gulf Cooperation Council': 'RME',
 'IAI Area, North America': 'RNA',
 'IAI Area, Russia & RER w/o EU27 & EFTA': 'RU',
 'IAI Area, South America': 'RLA',
 'NORDEL': 'ENTSO-E',
 'North America without Quebec': 'RNA',
 'OCEANIA': 'UN-OCEANIA',
 'RER w/o CH+DE': 'RER',
 'RER w/o DE+NL+RU': 'RER',
 'RER w/o RU': 'RER',
 'RoE': 'RER',
 'RoW': 'GLO',
 'SAS': 'RAS',
 'UCTE without Germany': 'ENTSO-E',
 'UCTE': 'ENTSO-E',
 'WECC': 'US',
 'WEU': 'RER',
 'IN-Eastern grid': 'IN',
 'IN-DD': 'IN',
 'BR-PA': 'BR',
 'CN-SA': 'CN',
 'CN-XJ': 'CN',
 'CN-BJ': 'CN',
 'BR-ES': 'BR',
 'BR-Southern grid': 'BR',
 'IN-KA': 'IN',
 'CN-HA': 'CN',
 'CA-NF': 'CA',
 'IN-WB': 'IN',
 'CN-HN': 'CN',
 '

In [11]:
bd.databases['ecoinvent-3.10-cutoff']["location_normalization"] = mapping
bd.databases.flush()

In [12]:
bd.Database('ecoinvent-3.10-cutoff').process()

In [9]:
list(bd.databases)

['ecoinvent-3.8-biosphere',
 'ecoinvent-3.8-cutoff',
 'ecoinvent-3.10-biosphere',
 'ecoinvent-3.10-cutoff',
 'generic-regionalized-biosphere-proxies',
 'generic_simapro_ecoinvent_biosphere_placeholders',
 'agribalyse-3.1.1-biosphere-placeholders',
 'agribalyse-3.1.1']

In [5]:
{x['location'] for x in bd.Database("agribalyse-3.1.1")}

{'AR',
 'AU',
 'Asia, South East',
 'BR',
 'CA',
 'CI',
 'CM',
 'CN',
 'CO',
 'CR',
 'DE',
 'EC',
 'ES',
 'Europe, Eastern',
 'Europe, Western',
 'FR',
 'FSU',
 'GB',
 'GLO',
 'IE',
 'IL',
 'IT',
 'KE',
 'MX',
 'MY',
 'Mixed data',
 'NO',
 None,
 'PE',
 'PH',
 'RAF',
 'RER',
 'RLA',
 'RME',
 'RNA',
 'RU',
 'SAS',
 'South and Central America',
 'TH',
 'UA',
 'US',
 'World'}

In [3]:
url = "https://wulca-waterlca.org/wordpress/wp-content/uploads/AWARE_watersheds_v12_20240531.xlsx"

Signature: tempfile.mkdtemp(suffix=None, prefix=None, dir=None)
Docstring:
User-callable function to create and return a unique temporary
directory.  The return value is the pathname of the directory.

Arguments are as for mkstemp, except that the 'text' argument is
not accepted.

The directory is readable, writable, and searchable only by the
creating user.

Caller is responsible for deleting the directory when done with it.
File:      /opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/tempfile.py
Type:      function